In [1]:
from vanna.openai import OpenAI_Chat
from vanna.vannadb import VannaDB_VectorStore
import os
from dotenv import load_dotenv
load_dotenv()

class MyVanna(VannaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        MY_VANNA_MODEL = 'elie'
        VannaDB_VectorStore.__init__(self, vanna_model=MY_VANNA_MODEL, vanna_api_key='02d3fb738a4c42de8b2ca3ec356450c0', config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key':'os.getenv('OPENAI_API_KEY')', 'model': 'gpt-4o-mini'})
# vn.remove_training_data(id='0')



odbc_conn_str = f"""DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={os.getenv('DB_SERVER')};DATABASE={os.getenv('DB_DATABASE')};UID={os.getenv('DB_USER')};PWD={os.getenv('DB_PASSWORD')};TrustServerCertificate={os.getenv('DB_TRUST_SERVER_CERTIFICATE')}"""

vn.connect_to_mssql(odbc_conn_str=odbc_conn_str)


In [2]:
response = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")
print(response)  # Afficher la réponse brute
print(type(response))  # Pour voir si c'est bien un objet contenant du JSON



       TABLE_CATALOG TABLE_SCHEMA              TABLE_NAME         COLUMN_NAME  \
0     3E_80_Commande          dbo  Mahn_Mahnbestandposten                  ID   
1     3E_80_Commande          dbo  Mahn_Mahnbestandposten          MahnlaufID   
2     3E_80_Commande          dbo  Mahn_Mahnbestandposten       MahnbestandID   
3     3E_80_Commande          dbo  Mahn_Mahnbestandposten          Mahnsperre   
4     3E_80_Commande          dbo  Mahn_Mahnbestandposten  MahnDatumBisSperre   
...              ...          ...                     ...                 ...   
5832  3E_80_Commande          dbo            D_Produktion             Element   
5833  3E_80_Commande          dbo            D_Produktion             Fluegel   
5834  3E_80_Commande          dbo            D_Produktion              Nummer   
5835  3E_80_Commande          dbo            D_Produktion               Wagen   
5836  3E_80_Commande          dbo            D_Produktion                Fach   

      ORDINAL_POSITION COLU

In [ ]:
# # Récupérer les métadonnées du schéma
# # df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")
# df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME IN ('P_Zubeh', 'A_Kopf', 'P_Kaufm', 'A_Vorgang', 'P_Teil', 'P_Artikel', 'P_Werkstoffe', 'A_KopfFreie', 'A_Adresse')")



# # Générer le plan d'entraînement
# plan = vn.get_training_plan_generic(df_information_schema)
# print(plan)

# # Entraîner le modèle
# vn.train(plan=plan)


In [ ]:
# vn.train(
#     question="""Combien de fois un article a-t-il été utilisé, et quelles sont ses dimensions totales en mL pour l'année 2024 ?""",
#     sql="""
#     SELECT 
#         Paramgen.ArtikelID AS [Article],
#         COUNT(Cde.AuNummer) AS [Nbr d'utilisations],
#         SUM(Paramgen.Dim1) / 1000 AS [Dim 1 (mL)]

#     FROM dbo.P_Artikel AS Paramgen                
#     LEFT JOIN dbo.A_Kopf AS Cde ON Cde.ID = Paramgen.ID_A_Kopf
#     LEFT JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf

#     WHERE 
#         cf.Feld3 BETWEEN '2024-01-01' AND '2024-12-31'
#         AND Paramgen.ArtikelID = '2104'
#     GROUP BY Paramgen.ArtikelID;
#     """
# )

# vn.train(
#     question="liste moi les 10 dernieres commandes avec leur Extensions ?", 
#     sql="""SELECT TOP 10 
#             Cde.AuNummer as [Numéro de commande], Cde.AuAlpha as [Extension], Cde.AufStatus as [Statut], Convert(date, cf.Feld3) as [Date entrée de commande]
#         FROM A_kopf as Cde
#         LEFT JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf
#         ORDER BY cf.Feld3 DESC"""
# )

# vn.train(
#     question="Liste des commandes où l'article 6101 est présent par extension, position et quantité entre en décembre 2024 ?",
#     sql="""
#         SELECT
#             Paramgen.ArtikelID AS [Article],
#             CONVERT(VARCHAR, Cde.AuNummer) AS [N° de Cde],
#             Cde.AuAlpha AS Extension,
#             Paramgen.Position AS [Position],
#             pk.Stueck AS Qté,        
#             COUNT(Paramgen.ArtikelID) * pk.Stueck AS [Nbr d'utilisations]
#         FROM 
#             dbo.P_Artikel AS Paramgen                
#             LEFT JOIN dbo.A_Kopf AS Cde ON Cde.ID = Paramgen.ID_A_Kopf
#             LEFT JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf

#             LEFT JOIN dbo.P_Kaufm AS pk ON Cde.ID = pk.ID_A_Kopf AND Paramgen.Position = pk.Position 
#         WHERE 
#             cf.Feld3 BETWEEN '2024-12-01' AND '2024-12-31'
#             AND Paramgen.ArtikelID = '6101'
#         GROUP BY
#             Cde.AuNummer,
#             Paramgen.ArtikelID,
#             Cde.AuAlpha,
#             pk.Stueck,
#             Paramgen.Position
#     """
# )
# vn.train(
#     question="Combien de châssis sont en préplanification ?", 
#     sql="""
#     SELECT 
#         SUM(CAST(NM.Feld2 AS INT)) AS [Total Châssis en Préplanification]
#     FROM 
#         dbo.A_Kopf AS Cde
#         INNER JOIN dbo.A_KopfFreie AS NM ON Cde.ID = NM.ID_A_Kopf
#     WHERE 
#         Cde.AufStatus = 'pré planif'
#         AND CAST(NM.Feld2 AS INT) > 0
#         AND Cde.AuAlpha NOT LIKE '%N%'
#         AND Cde.AuAlpha NOT LIKE '%S%'
#         AND Cde.AuAlpha NOT LIKE '%J%'
#         AND Cde.AuAlpha NOT LIKE '%ECH%'
#         AND Cde.AuAlpha NOT LIKE '%PFI%'
#         AND Cde.AuAlpha NOT LIKE '%MKG%'
#         AND Cde.AuAlpha NOT LIKE '%9%'
#         AND Cde.AuAlpha NOT LIKE '%MUR%'
# """)

# vn.train(
#     question="Quelle est la somme des chassis en pré-planif ?", 
#     sql="""
#     SELECT 
#         SUM(TotalPreplanif) AS [Total de préplanif]
#     FROM (
#         SELECT
#             COUNT(1) * NM.Feld2 AS TotalPreplanif
#         FROM 
#             dbo.A_Kopf AS Cde
#             INNER JOIN dbo.A_KopfFreie AS NM ON Cde.ID = NM.ID_A_Kopf
#         WHERE 
#             Cde.AufStatus = 'pré planif'
#             AND Cde.AuAlpha NOT LIKE '%N%'
#             AND Cde.AuAlpha NOT LIKE '%S%'
#             AND Cde.AuAlpha NOT LIKE '%J%'
#             AND Cde.AuAlpha NOT LIKE '%ECH%'
#             AND Cde.AuAlpha NOT LIKE '%PFI%'
#             AND Cde.AuAlpha NOT LIKE '%MKG%'
#             AND Cde.AuAlpha NOT LIKE '%9%'
#             AND Cde.AuAlpha NOT LIKE '%MUR%'
#             AND NM.Feld2 > 0
#         GROUP BY NM.Feld2
#     ) AS Subquery
#     """
# )
# vn.train(
#     question="Combien de chassis en pré-planif ?", 
#     sql="""
#     SELECT 
#     SUM(CAST(NM.Feld2 AS INT)) AS [Total Châssis en Préplanif]
# FROM 
#     dbo.A_Kopf AS Cde
#     INNER JOIN dbo.A_KopfFreie AS NM ON Cde.ID = NM.ID_A_Kopf
# WHERE 
#     Cde.AufStatus = 'pré planif'
#     AND CAST(NM.Feld2 AS INT) > 0;
#     """
# )

# vn.train(
#     question="liste les commandes en préplanif", 
#     sql="""
#     WITH LatestOrders AS (
#         SELECT
#             CONVERT(VARCHAR, Cde.AuNummer) AS [N° de Cde],
#             CONVERT(VARCHAR, Cde.AuAlpha) AS [Extension],
#             CONVERT(INT, NM.Feld2) AS [Nbr Menuiserie],
#             Cde.Kommission as [Affaire],
#             ADR.Firma as [Client],
#             Cde.Techniker as [Technicien],
#             DATEPART(iso_week, DateSerie.Datum) AS [Semaine entrée cde],
#             DATEPART(iso_week, NM.Feld5) AS [Semaine Livraison],
#             Cde.AufStatus as [Status],
#             DATEDIFF(DAY, DateSerie.Datum, GETDATE()) AS [Diff jours - Date entrée],
#             DATEDIFF(DAY, NM.Feld5, GETDATE()) AS [Diff jours - Livraison],
#             ROW_NUMBER() OVER (PARTITION BY Cde.AuNummer, Cde.AuAlpha ORDER BY DateSerie.Datum DESC) as RowNum
#         FROM 
#             dbo.A_Kopf AS Cde
#             LEFT JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf
#             INNER JOIN dbo.A_Adresse AS ADR ON Cde.ID = ADR.ID_A_Kopf
          
#         WHERE 
#             Cde.AufStatus IN ('Pré planif')
#             AND Cde.AuAlpha NOT LIKE '%N%'
#             AND Cde.AuAlpha NOT LIKE '%S%'
#             AND Cde.AuAlpha NOT LIKE '%J%'
#             AND Cde.AuAlpha NOT LIKE '%ECH%'
#             AND Cde.AuAlpha NOT LIKE '%PFI%'
#             AND Cde.AuAlpha NOT LIKE '%MKG%'
#             AND Cde.AuAlpha NOT LIKE '%9%'
#             AND Cde.AuAlpha NOT LIKE '%MUR%'
#             AND NM.Feld2 > 0
#             AND DateSerie.Nummer LIKE '%entrée cde%'
#             AND DateSerie.Nummer NOT LIKE '%_CdeModifiée%'
#             AND DateSerie.Nummer NOT LIKE '%_Mess. Client%'
#             AND ADR.Firma NOT LIKE ''
#     )
#     SELECT 
#         [N° de Cde],
#         [Extension],
#         [Nbr Menuiserie],
#         [Affaire],
#         [Client],
#         [Technicien],
#         [Semaine entrée cde],
#         [Semaine Livraison],
#         [Status],
#         [Diff jours - Date entrée],
#         [Diff jours - Livraison]
#     FROM LatestOrders
#     WHERE RowNum = 1
#     """
# )

# vn.train(
#     question="Combien de chassis sont entrés aujourd'hui ?", 
#     sql="""
#     SELECT 
#         SUM([Nbr de chassis entrés aujourd'hui]) AS [Nbr de chassis entrés aujourd'hui]
#     FROM (
#         SELECT
#             COUNT(1) * cf.Feld2 AS [Nbr de chassis entrés aujourd'hui]
#         FROM
#             dbo.A_Kopf AS Cde
#             LEFT JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf
#         WHERE
#             CONVERT(DATE, cf.Feld3) = CONVERT(DATE, GETDATE())
#             AND Cde.AuAlpha NOT LIKE '%N%'
#             AND Cde.AuAlpha NOT LIKE '%S%'
#             AND Cde.AuAlpha NOT LIKE '%J%'
#             AND Cde.AuAlpha NOT LIKE '%ECH%'
#             AND Cde.AuAlpha NOT LIKE '%PFI%'
#             AND Cde.AuAlpha NOT LIKE '%MKG%'
#             AND Cde.AuAlpha NOT LIKE '%9%'
#             AND Cde.AuAlpha NOT LIKE '%MUR%'
#         GROUP BY cf.Feld2
#     ) AS Subquery
# """
# )
# vn.train(
#     question="Combien de chassis ont été saisies aujourd'hui ?", 
#     sql="""
#     SELECT 
#     SUM(CAST(NM.Feld2 AS INT)) AS [Total Châssis Saisis Aujourd'hui]
# FROM 
#     dbo.A_Kopf AS Cde
#     INNER JOIN dbo.A_Vorgang AS DateSerie ON DateSerie.ID_A_Kopf = Cde.ID
#     INNER JOIN dbo.A_KopfFreie AS NM ON Cde.ID = NM.ID_A_Kopf
# WHERE 
#     CONVERT(DATE, DateSerie.Datum) = CONVERT(DATE, GETDATE())
#     AND DateSerie.Nummer LIKE '%entrée cde%'
#     AND Cde.AuAlpha NOT LIKE '%N%'
#     AND Cde.AuAlpha NOT LIKE '%S%'
#     AND Cde.AuAlpha NOT LIKE '%J%'
#     AND Cde.AuAlpha NOT LIKE '%ECH%'
#     AND Cde.AuAlpha NOT LIKE '%PFI%'
#     AND Cde.AuAlpha NOT LIKE '%MKG%'
#     AND Cde.AuAlpha NOT LIKE '%9%'
#     AND Cde.AuAlpha NOT LIKE '%MUR%'
#     """
# )
# vn.train(
#     question="Combien de chassis ont été saisies aujourd'hui par technicien ?", 
#     sql="""
#     SELECT
#         Cde.Techniker AS [Technicien],
#         SUM(CASE 
#             WHEN Cde.AufStatus = 'BLOQ CC' AND DateSerie.Nummer LIKE '%q CC%' THEN cf.Feld2 
#             ELSE 0 
#         END) AS [total de BLOQ CC],
#         SUM(CASE 
#             WHEN Cde.AufStatus = 'BLOQ saisie' AND DateSerie.Nummer LIKE '%Impr%' THEN cf.Feld2 
#             ELSE 0 
#         END) AS [total de BLOQ saisie],
#         SUM(CASE 
#             WHEN Cde.AufStatus = 'BLOQ CC' AND DateSerie.Nummer LIKE '%q CC%' THEN Cde.A_VorMwSt 
#             WHEN Cde.AufStatus = 'BLOQ saisie' AND DateSerie.Nummer LIKE '%Impr%' THEN Cde.A_VorMwSt 
#             ELSE 0 
#         END) AS [Prix HT Total]
#     FROM 
#         dbo.A_Kopf AS Cde
#         INNER JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf
#         INNER JOIN dbo.A_Vorgang AS DateSerie ON DateSerie.ID_A_Kopf = Cde.ID
#     WHERE 
#         Cde.Techniker IS NOT NULL
#         AND Cde.Techniker NOT LIKE '%cindy%'
#         AND Cde.Techniker NOT LIKE '%morgan%'
#         AND Cde.Techniker NOT LIKE '%AURELIE MASSET%'
#         AND Cde.Techniker NOT LIKE '%MANUELDAEVIDOZO%'
#         AND Cde.Techniker NOT LIKE '%MELANIE SANDRE%'
#         AND Cde.Techniker NOT LIKE '%JEROME%'
#         AND Cde.Techniker NOT LIKE '%Antoine%'
#         AND Cde.Techniker NOT LIKE '%GEORGE HENRI%'
#         AND Cde.AufStatus IN ('BLOQ CC', 'BLOQ saisie')
#         AND CONVERT(DATE, DateSerie.Datum) = CONVERT(DATE, GETDATE())
#     GROUP BY Cde.Techniker
#     """
# )
# vn.train(
#     question="Liste moi les commandes bloquées aujourd'hui", 
#     sql="""
#     SELECT
#         CONVERT(VARCHAR, Cde.AuNummer) AS [N° de Cde],
#         CONVERT(VARCHAR, Cde.AuAlpha) AS [Extension],
#         CONVERT(INT, cf.Feld2) AS [Nbr Menuiserie],
#         Cde.AufStatus as [Status],
#         Cde.Techniker AS [Technicien]
#     FROM 
#         dbo.A_Kopf AS Cde
#         INNER JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf
#         INNER JOIN dbo.A_Vorgang AS DateSerie ON DateSerie.ID_A_Kopf = Cde.ID
#     WHERE 
#         Cde.Techniker IS NOT NULL
#         AND Cde.AufStatus IN ('BLOQ CC', 'BLOQ saisie')
#         AND CONVERT(DATE, DateSerie.Datum) = CONVERT(DATE, GETDATE())
#     """
# )
# vn.train(
#     question="Liste moi toutes les commandes bloquées", 
#     sql="""
#         SELECT
#             CONVERT(VARCHAR, Cde.AuNummer) AS [N° de Cde],
#             CONVERT(VARCHAR, Cde.AuAlpha) AS [Extension],
#             CONVERT(INT, NM.Feld2) AS [Nbr Menuiserie],
#             ADR.Firma as [Client],
#             Cde.AufStatus as [Status]
#         FROM 
#             dbo.A_Kopf AS Cde
#             INNER JOIN dbo.A_KopfFreie AS NM ON Cde.ID = NM.ID_A_Kopf
#             INNER JOIN dbo.A_Adresse AS ADR ON Cde.ID = ADR.ID_A_Kopf
#         WHERE 
#             Cde.AufStatus LIKE '%bloq%'
#             AND ADR.Firma IS NOT NULL
#             AND ADR.Firma NOT LIKE ''

#         GROUP BY
#             Cde.AuNummer,
#             Cde.AuAlpha,
#             Cde.AufStatus,
#             NM.Feld2,
#             ADR.Firma,
#             Cde.Techniker,
#             NM.Feld5
#         ORDER BY 
#             Cde.AufStatus ASC
#         """
# )
# vn.train(
#     question="liste moi les commandes de février 2025 avec un montant ht supérieur à 2000 €", 
#     sql="""SELECT 
#         Cde.AuNummer AS [N° de Cde],
#         Cde.AuAlpha AS [Extension],
#         SUM(Cde.A_VorMwSt) AS [Montant HT],
#         CONVERT(DATE, cf.Feld3) AS [Date de commande]
#     FROM 
#         dbo.A_Kopf AS Cde
#         LEFT JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf
#     WHERE 
#         CONVERT(DATE, cf.Feld3) BETWEEN '2025-02-01' AND '2025-03-01'
#         AND Cde.A_VorMwSt > 2000
#     GROUP BY 
#         Cde.AuNummer, Cde.AuAlpha, cf.Feld3"""
# )
# vn.train(
#     question="liste moi les commandes de février 2025 avec un montant ht supérieur à 1000 €", 
#     sql="""SELECT 
#         Cde.AuNummer AS [N° de Cde],
#         Cde.AuAlpha AS [Extension],
#         SUM(Prix.VkNRabMalStk) AS [Montant HT],
#         CONVERT(DATE, cf.Feld3) AS [Date entrée de commande]
#         FROM 
#                 dbo.A_Kopf AS Cde
#                 LEFT JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf
#                 INNER JOIN dbo.P_Kaufm AS Prix ON Cde.ID = Prix.ID_A_Kopf
#         WHERE 
#                 CONVERT(DATE, cf.Feld3) >= '2025-02-01' 
#                 AND CONVERT(DATE, cf.Feld3) < '2025-03-01'
#         GROUP BY 
#                 Cde.AuNummer, Cde.AuAlpha, cf.Feld3
#         HAVING 
#                 SUM(Prix.VkNRabMalStk) > 1000"""
# )
# vn.train(
#     question="Donne le nombre d'utilisation et le montant HT de la variante 224 au premier trimestre 2025", 
#     sql="""
#     SELECT
#         Variante.Variante AS [Variante],
#         SUM(Prix.Stueck) AS [Nbr d'utilisations],
#         SUM(Prix.VkNRabMalStk) AS [CA HT position],
#         CONVERT(DATE, cf.Feld3) AS [Date de commande]
#     FROM 
#         dbo.P_Teil AS Variante
#         INNER JOIN dbo.A_Kopf AS Cde ON Cde.ID = Variante.ID_A_Kopf
#         INNER JOIN dbo.P_Kaufm AS Prix ON Cde.ID = Prix.ID_A_Kopf AND Variante.Position = Prix.Position
#         LEFT JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf
#     WHERE 
#         Variante.Variante = '224'
#         AND Prix.Position > 0
#         AND cf.Feld3 BETWEEN '2025-01-01' AND '2025-03-31'
#     GROUP BY
#         Variante.Variante, cf.Feld3
#     ORDER BY
#         [Date de commande]"""
# )
# vn.train(
#     question="Liste moi les commandes des 30 derniers jours contenant l'article 224 avec le status facturée", 
#     sql="""
#     SELECT
#         CONVERT(VARCHAR, Cde.AuNummer) AS [N° de Cde],
#         CONVERT(VARCHAR, Cde.AuAlpha) AS [Extension],
#         Variante.Variante AS [Variante],
#         CONVERT(DATE, cf.Feld3) AS [Date entrée cde],
#         Cde.AufStatus as [Status]
#     FROM 
#         dbo.P_Teil AS Variante
#         INNER JOIN dbo.A_Kopf AS Cde ON Cde.ID = Variante.ID_A_Kopf
#         INNER JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf
#     WHERE 
#         CONVERT(DATE, cf.Feld3) BETWEEN DATEADD(DAY, -30, GETDATE()) AND GETDATE()
#         AND Cde.AufStatus IN ('facturée')
#         AND Variante.Variante = '224'
#     GROUP BY
#         CONVERT(VARCHAR, Cde.AuNummer),
#         CONVERT(VARCHAR, Cde.AuAlpha),
#         Variante.Variante,
#         CONVERT(DATE, cf.Feld3),
#         CAST(YEAR(cf.Feld3) AS INT),
#         CAST(MONTH(cf.Feld3) AS INT),
#         cf.Feld3,
#         Cde.AufStatus
#     ORDER BY
#         [Date entrée cde] DESC
#     """
# )
# vn.train(
#     question="trouve moi les numero de commande et extension qui contiennent l'article SBST M AUWS 28 qui n'ont pas le status facturée", 
#     sql=""" 
#         SELECT DISTINCT 
#             Cde.AuNummer AS [N° de Cde],
#             Cde.AuAlpha AS [Extension]
#         FROM 
#             dbo.A_Kopf AS Cde
#             INNER JOIN dbo.P_Artikel AS Paramgen ON Cde.ID = Paramgen.ID_A_Kopf
#         WHERE 
#             Paramgen.ArtikelID = 'SBST M AUWS 28'
#             AND Cde.AufStatus <> 'facturée'
#     """
# )
# vn.train(
#     question="trouve moi les numero de commande et extension avec la date de commande soubassement qui contiennent le soubassement code article SBST M AUWS 28 qui n'ont pas le status facturée", 
#     sql=""" 
#         SELECT DISTINCT 
#             Cde.AuNummer AS [N° de Cde],
#             Cde.AuAlpha AS [Extension],
#             cf.Feld1 AS [Date de commande soubassement]
#         FROM 
#             dbo.A_Kopf AS Cde
#             INNER JOIN dbo.P_Artikel AS Paramgen ON Cde.ID = Paramgen.ID_A_Kopf
#             LEFT JOIN dbo.A_KopfFreie AS cf ON Cde.ID = cf.ID_A_Kopf

#         WHERE 
#             Paramgen.ArtikelID = 'SBST M AUWS 28'
#             AND Cde.AufStatus <> 'facturée'
#     """
# )


In [ ]:
# training_data = vn.get_training_data()
# training_data



In [ ]:
# reponse = vn.ask("liste moi les 10 dernieres commandes ?")
# print(reponse)


In [7]:
from vanna.flask import VannaFlaskApp
VannaFlaskApp(vn, allow_llm_to_see_data=True).run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
None
Using model gpt-4o-mini for 14808.0 tokens (approx)
Using model gpt-4o-mini for 93.5 tokens (approx)
Using model gpt-4o-mini for 109.75 tokens (approx)
None
Using model gpt-4o-mini for 13441.0 tokens (approx)
Using model gpt-4o-mini for 111.75 tokens (approx)
